In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split



## DATA


In [2]:
def load_data():
    ## preparing data
    # Prefix
    target_image_width = 160
    target_image_height = 130
    # parent dataset
    dataset_parent_folder = 'dataset'
    
    if not (os.path.exists(dataset_parent_folder) and os.path.isdir(dataset_parent_folder)):
        print(f"Error: Cant find the path {dataset_parent_folder}")
        raise FileNotFoundError(f"Dataset folder could not be found at {dataset_parent_folder}")
    
    print(f"Successfully accessed the parent dataset folder: {dataset_parent_folder}")
    
    # Subdirectories
    all_entries = os.listdir(dataset_parent_folder)
    # This line ensures all_dog_breed contains only names of actual directories, sorted
    all_dog_breed = sorted([entry for entry in all_entries if os.path.isdir(os.path.join(dataset_parent_folder, entry)) 
                                                            and not entry.startswith(".") ])
    
    if not all_dog_breed: # Check if the list is empty
        print(f"Warning: No breed subdirectories found in '{dataset_parent_folder}'.")
        raise FileNotFoundError("No breed subdirectories found to process.")
    else:
        print(f"Identified breed folders: {all_dog_breed}")
    
    num_of_breed = len(all_dog_breed) # Define num_of_breed
    print(f"Number of classes (breeds): {num_of_breed}")
    
    # one_hot_setup
    breed_to_idx = {breed_name:i for i, breed_name  in enumerate(all_dog_breed)}
    idx_to_breed = {i: breed_name for i, breed_name in enumerate(all_dog_breed)} # Useful for verification
    print(f"Breed to index mapping: {breed_to_idx}")
    
    # X Y (Initialize lists for all data)
    x_final = []
    y_final = []
    
    # Loop through the identified breed folder names
    for current_breed in all_dog_breed:
        current_breed_path = os.path.join(dataset_parent_folder, current_breed)
        print(f"\nProcessing breed: {current_breed}")
    
        # 1-hot-for the current breed
        one_hot_for_current_breed = np.zeros(num_of_breed, dtype=int) # Use dtype=int for one-hot
        current_breed_index = breed_to_idx[current_breed]
        one_hot_for_current_breed[current_breed_index] = 1
    
        files_in_current_breed = os.listdir(current_breed_path)
        loaded_count = 0 # To count images loaded for the current breed
    
        for file_name in files_in_current_breed:
            image_file_path = os.path.join(current_breed_path, file_name)
    
            # Check if the file is an image based on its extension
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
                try:
                    # Open the image using Pillow
                    img = Image.open(image_file_path)
    
                    # Convert to RGB format for consistency
                    img = img.convert('RGB')
    
                    # Resize the image if it's not the target size
                    if img.size != (target_image_width, target_image_height):
                        img = img.resize((target_image_width, target_image_height))
    
                    # Append the image data (as NumPy array) and its one-hot label
                    x_final.append(np.array(img)) # Appending NumPy array of the image
                    y_final.append(one_hot_for_current_breed)
                    loaded_count +=1
    
                except Exception as e:
                    print(f"    Error loading or processing image '{image_file_path}': {e}")
        print(f"  Loaded {loaded_count} images for {current_breed}.")
    
    
    # Convert x y to NumPy arrays for easier use with ML frameworks
    if x_final and y_final: # Check if lists are not empty
        X_final_np_4D = np.array(x_final) # This creates the 4D array (num_images, height, width, channels)
        Y_final_np_2D = np.array(y_final) # This creates the 2D array (num_images, num_of_breed)
    
        print("\n--- Initial Array Shapes ---")
        print(f"Shape of X_final_np_4D (image data): {X_final_np_4D.shape}")
        print(f"Shape of Y_final_np_2D (labels): {Y_final_np_2D.shape}")
    
        # --- Reshaping X data for a dense layer (flattening each image) ---
        num_of_images = X_final_np_4D.shape[0] # Or simply len(x_final)
    
        # IMPORTANT CORRECTION: reshape() returns a new array (or a view).
        # You need to assign the result if you want to use the reshaped array.
        X_final_np_flattened = X_final_np_4D.reshape(num_of_images, -1) # -1 infers the flattened dimension size
    
        print("\n--- Reshaped Array Shape (for Dense Layer Input) ---")
        print(f"Shape of X_final_np_flattened (flattened image data): {X_final_np_flattened.shape}") # Should be (num_images, height*width*channels)
    
        # Now you have two versions of X:
        # X_final_np_4D: If you want to use CNNs (shape: num_images, height, width, channels)
        # X_final_np_flattened: If you want to use a simple dense layer (shape: num_images, features)
    
        # Print example of first few data points for verification
        if len(X_final_np_flattened) > 0: # Ensure there's data to print
            print("\nExample of first few loaded data points (using flattened X):")
            for i in range(min(5, num_of_images)):
                # np.argmax(Y_final_np_2D[i]) gives the integer index of the breed
                breed_index = np.argmax(Y_final_np_2D[i])
                print(f"  Flattened Image {i} feature count: {X_final_np_flattened[i].shape[0]}, Label (one-hot): {Y_final_np_2D[i]}, Breed Name: {idx_to_breed[breed_index]}")
    
    else:
        print("\n--- Image Loading Process Complete ---")
        print("No images were loaded, or no labels were created.")
    return X_final_np_flattened, Y_final_np_2D


In [3]:
X, Y = load_data()
X_train,X_test,Y_train, Y_test = train_test_split(X,Y,test_size = 0.4,random_state = 42)
X_test,X_validation,Y_test,Y_validation = train_test_split(X_test,Y_test,test_size = 0.5,random_state = 42)

m_train = X_train.shape[0]
# Define input size and number of classes from data
input_size_train = X_train.shape[1]         # Number of features in flattened images
num_classes_train = Y_train.shape[1]        # Number of output classes (from one-hot Y_train)
print(X_train.shape,X_test.shape,X_validation.shape)
print(Y_train)


Successfully accessed the parent dataset folder: dataset
Identified breed folders: ['Beagle', 'Boxer', 'Dachshund', 'German_Shepherd', 'Golden_Retriever', 'Labrador_Retriever', 'Poodle', 'Rottweiler', 'Yorkshire_Terrier']
Number of classes (breeds): 9
Breed to index mapping: {'Beagle': 0, 'Boxer': 1, 'Dachshund': 2, 'German_Shepherd': 3, 'Golden_Retriever': 4, 'Labrador_Retriever': 5, 'Poodle': 6, 'Rottweiler': 7, 'Yorkshire_Terrier': 8}

Processing breed: Beagle
  Loaded 100 images for Beagle.

Processing breed: Boxer
  Loaded 100 images for Boxer.

Processing breed: Dachshund
  Loaded 96 images for Dachshund.

Processing breed: German_Shepherd
  Loaded 96 images for German_Shepherd.

Processing breed: Golden_Retriever
  Loaded 91 images for Golden_Retriever.

Processing breed: Labrador_Retriever
  Loaded 95 images for Labrador_Retriever.

Processing breed: Poodle
  Loaded 100 images for Poodle.

Processing breed: Rottweiler
  Loaded 89 images for Rottweiler.

Processing breed: Yorksh

In [4]:
## --- 3. Neural Network Custom Functions ---

# Activation: ReLU
def relu(Z):
    return np.maximum(0, Z)

# Activation: Linear (Identity)
def linear(Z):
    return Z

# Activation: Softmax (Numerically Stable)
def softmax(Z):
    Z_stabilized = Z - np.max(Z, axis=1, keepdims=True) # Stability trick
    exp_Z = np.exp(Z_stabilized)
    sum_exp_Z = np.sum(exp_Z, axis=1, keepdims=True)
    probabilities = exp_Z / sum_exp_Z
    return np.nan_to_num(probabilities) # Handle potential NaN from 0/0



In [5]:
## --- 4. Neural Network Setup ---

# Define layer architecture
dense1_units = 30
dense2_units = 20
output_units = num_classes_train

# Initialize Weights and Biases
# Using small random numbers for weights and zeros for biases
W1 = np.random.randn(input_size_train, dense1_units) * 0.01
b1 = np.zeros((1, dense1_units))
W2 = np.random.randn(dense1_units, dense2_units) * 0.01
b2 = np.zeros((1, dense2_units))
W3 = np.random.randn(dense2_units, output_units) * 0.01
b3 = np.zeros((1, output_units))

print("\n--- Weight and Bias Shapes ---")
print(f"W1: {W1.shape}, b1: {b1.shape}")
print(f"W2: {W2.shape}, b2: {b2.shape}")
print(f"W3: {W3.shape}, b3: {b3.shape}")


--- Weight and Bias Shapes ---
W1: (62400, 30), b1: (1, 30)
W2: (30, 20), b2: (1, 20)
W3: (20, 9), b3: (1, 9)


## Foward


In [ ]:
# Dense Layer Computation
def my_dense(A_in, W, b, g_activation): # Renamed 'g' to 'g_activation' for clarity
    # A_in: (m, n_features_in), W: (n_features_in, n_units_out), b: (1, n_units_out)
    Z = np.matmul(A_in, W) + b  # Linear transformation
    A_out = g_activation(Z)     # Apply activation function
    return A_out,Z

# Sequential Model (3-layer feedforward)
def my_sequential(X, W1,W2,W3,b1,b2,b3,g1,g2,g3):
    A1,Z1 = my_dense(X, W1, b1, g1)      # First hidden layer
    A2,Z2 = my_dense(A1, W2, b2, g2)     # Second hidden layer
    A3,Z3 = my_dense(A2, W3, b3, g3)     # Output layer
    return A1,Z1,A2,Z2,A3,Z3
def compute_forward(X,W1,W2,W3,b1,b2,b3):
    Z_logits, cache = my_sequential(X, W1, b1, relu, W2, b2, relu, W3, b3, linear)
    print(Z_logits.max())

A1: (520, 30)
A2: (520, 20)
A3: (520, 9)
Z1: (520, 30)
Z2: (520, 20)
Z_LOGITS: (520, 9)


## Lost function



In [7]:
def cal_log_softmax(Z_logits):
    Z_logits_stabilized = Z_logits - np.max(Z_logits, axis=1, keepdims=True) # Stability trick
    e_Z = np.exp(Z_logits_stabilized)
    sum_e_Z = np.sum(e_Z, axis =1, keepdims =True)
    epsilon = 1e-9 # Small constant to prevent log(0)
    log_softmax_value = np.log(e_Z / (sum_e_Z + epsilon) +epsilon) # Add epsilon to denominator too for 0/0 cases
                                                                          # or rely on nan_to_num after division
    return log_softmax_value
def cal_cost(Z_logits, Y):
    log_softmax_val = cal_log_softmax(Z_logits)
    sample_wise_loss = -np.sum(Y*log_softmax_val, axis=1)
    cost= np.mean(sample_wise_loss)
    return cost

## Errors (dZ)


In [26]:
def cal_errors(Z1,Z2,Z_logits):
    def dRelu(Z):
        return (Z>0).astype(int)
    #output layer error
    dZ3 = 1/m_train*(softmax(Z_logits)-Y_train)
    #layer2 error
    dRelu_Z2 = dRelu(Z2)
    dZ2=(dZ3@(W3.T))*dRelu_Z2
    #Layer1 error
    dRelu_Z1 = dRelu(Z1)
    dZ1= (dZ2@W2.T)*dRelu_Z1
    dZ1.shape
    return dZ1,dZ2,dZ3

(520, 30)

## Gradients w.r.t W (dW)

In [29]:
def cal_garadients(dZ1,dZ2,dZ3,A1,A2,X):
#weights
    dW3 = A2.T@dZ3
    dW2 = A1.T@dZ2
    dW1 = X.T@dZ1
    #biases
    db3= np.sum(dZ3, axis=0,keepdims=0)
    db2= np.sum(dZ2, axis=0,keepdims=0)
    db1= np.sum(dZ1, axis=0,keepdims=0)
    return dW1,dW2,dW3

## compute gradient descents

In [ ]:

def compute_gradient_descent(dW3,dW2,dW1,db1,db2,db3,learning_rate):
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    W3 = W3 - learning_rate * dW3
    b3 = b3 - learning_rate * db3
    return W1,W2,W3,b1,b2,b3

## train the model

In [ ]:
def train_model(X,Y,W1,W2,W3,b1,b2,b3,num_epochs,learning_rate):
    for i in range(num_epochs):
        ## forward
        A1,Z1,A2,Z2,A3,Z_logits = my_sequential(X,W1,W2,W3,b1,b2,b3,relu,relu,linear)
        ##errors
        dZ1,dZ2,dZ3= cal_errors(Z1,Z2,Z_logits)
        ##gradients
        dW1,dW2,dW3= cal_garadients(dZ1,dZ2,dZ3,A1,A2,X)
        #gradient descent
        W1,W2,W3,b1,b2,b3 = compute_gradient_descent(dW3,dW2,dW1,learning_rate)
    return W1,W2,W3,b1,b2,b3